In [43]:
from pyspark.sql import SparkSession, functions as F
from pyspark.ml import feature as H
# First lets reed the datasets
spark = (
    SparkSession.builder.appName("Data_Explorer")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

In [44]:
# Read all our tables 
customer_scaled = spark.read.parquet('../data/Normalised/customer_scaled/')
industry_scaled = spark.read.parquet('../data/Normalised/industry_scaled/')
merchant_scaled = spark.read.parquet('../data/Normalised/merchant_scaled/')
revenue_scaled = spark.read.parquet('../data/Normalised/revenue_scaled/')


In [45]:
from operator import add
from functools import reduce
# For each table we need to perform the following steps
def scroring_process(dataset):
    # first subtract 1 from each cell then square
    for col in dataset.columns[1:]:
        dataset = dataset.withColumn(col, F.pow((F.col(col) - 1),2))
    # sum the columns 
    new_df = dataset.withColumn('total',F.round(reduce(add, [F.col(x) for x in dataset.columns[1:]]), 4))
    return new_df


In [46]:
# Define our scaliong function
def feature_standardisation(dataset): 
    values = dataset.select(F.max('total').alias('high'), F.min('total').alias('low'))
    dataset = dataset.withColumn('total', F.round((F.col('total') - values.select('low').head()[0]) / (values.select('high').head()[0] - values.select('low').head()[0]), 4))
    return dataset.select('merchant_abn', 'total')

In [47]:
customer_scaled = scroring_process(customer_scaled)
industry_scaled = scroring_process(industry_scaled)
merchant_scaled = scroring_process(merchant_scaled)
revenue_scaled = scroring_process(revenue_scaled)

In [48]:
customer_scaled

merchant_abn,postcode_entropy,Customer_Growth_Avg,Total_Business_Days,total
34440496342,0.44689225,0.37002889000000005,0.99860049,1.8155
35344855546,0.7659750399999999,0.29430625,0.9894280900000001,2.0497
37935728745,0.10903204000000004,0.08111104000000004,1.0,1.1901
38700038932,0.94187025,0.33350625,0.9514051600000001,2.2268
38986645707,0.21492496,0.20142143999999998,0.9998000100000001,1.4161
41956465747,0.48608784000000005,0.3217158399999999,0.998001,1.8058
35079621278,0.17007376,0.34304449,0.9998000100000001,1.5129
35769589414,0.61183684,0.28815423999999995,0.996004,1.896
37470904997,0.25371369000000005,0.21687649,0.99960004,1.4702
39759375662,0.31438448999999996,0.33779343999999994,0.9994000900000001,1.6516


In [49]:
# rescale them 
customer_scaled = feature_standardisation(customer_scaled)
customer_scaled = customer_scaled.withColumnRenamed('total','Customer_score')
revenue_scaled = scroring_process(revenue_scaled)
revenue_scaled = revenue_scaled.withColumnRenamed('total','Revenue_score')
industry_scaled = scroring_process(industry_scaled)
industry_scaled = industry_scaled.withColumnRenamed('total','Industry_score')
merchant_scaled = scroring_process(merchant_scaled)
merchant_scaled = merchant_scaled.withColumnRenamed('total','Merchant_score')

In [50]:
# Finally join 
Full_scores = customer_scaled.join(revenue_scaled, on='merchant_abn', how='inner').join(industry_scaled, on='merchant_abn', how='inner').join(merchant_scaled, on='merchant_abn', how='inner')

In [52]:
# now scale and done 
Full_scores = scroring_process(Full_scores)

In [53]:
Full_scores

22/10/09 23:33:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


merchant_abn,Customer_score,Total_Revenue,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Growth_4,Growth_5,Growth_6,Growth_7,Growth_8,Growth_9,Revenue_Growth_Avg,Revenue_score,Portion_of_Total_Revenue,Portion_of_Total_Transactions,Average_Weighted_Revenue,survival_rate,Industry_score,postcode_entropy,Customer_Growth_Avg,Total_Business_Days,Merchant_score,total
12516851436,0.16516096,0.9998966955801654,2.354063362947056...,7.135920463268369...,9.387077750813639E19,1.592382109657036...,1.415270567888266E22,2.962326981006539...,1.901467352960560...,0.06560422108751847,0.2556298441417374,0.5530011928036459,0.6780183058788245,0.9441091194282173,0.7432275235261395,0.12399849700749825,1.47815716765936E25,0.0,0.0,1.0,0.0,4.0,0.47109405205332266,0.28001300635245313,0.999996082747356,0.12292036000000002,1.501718135409322E25
15613631617,0.011880999999999997,0.9999418382034351,9.347192815418478E14,3.361604889727114...,1.420193256041672...,1.632989216176492E20,1.066136900089155...,4.290391388278361E21,1.303113571912672...,0.16934395785950168,0.40059265632366753,0.62766290945822,0.740292397747083,0.8352114624021277,0.8861936814404523,0.21759428732883845,1.409015304454287...,0.0,0.0,1.0,0.0,4.0,0.911276754007776,0.3109552278319725,0.9996944070190683,0.5168172100000001,1.427580806411168...
19839532017,0.04297329000000002,0.9994921523016987,1.233406410499186...,3.209939321587516...,1.598465849930851...,1.439608019591654E24,1.083210397046373...,5.23912238732868E25,1.251250843844476...,0.19140625,0.37444572329203507,0.6749908044414974,0.7110167690585989,0.7993194223731048,0.9250301701483773,0.2253139252095186,1.461012618309702...,0.1435354318884218,0.48296797985057716,0.012989571028315982,0.8814532708812811,1.1994630400000004,0.7681706275876734,0.33030157226160917,0.9999501255437558,0.28270489000000004,1.480007727220319...
34440496342,0.12517444,0.999994884101724,4.421031411025119E10,2.777587631879158...,2.102476027943141...,6.930203298496181E15,4.772869743706716E16,1.459125835520842...,6.304650020759623...,0.14168402814369574,0.3259969484249643,0.8738541728835035,0.7321417712610064,0.8861965514218967,0.8228900661927842,0.23882054123929997,6.587963277974416...,0.0,0.0,1.0,0.0,4.0,0.48173568700089886,0.36377351768694804,0.999996082747356,0.13527683999999993,6.671279956222235...
35344855546,0.025217440000000018,0.9997041549631407,4.131094972148423E17,1.278355207433769...,2.726859062275868...,5.786200754269229E22,3.624111889531733...,1.944732533470578E24,8.987403344344514E24,0.17837464255389196,0.5113427486580417,0.5118348389283374,0.748112186579843,0.7789220694500912,0.8082675191243264,0.19245796986928482,6.708516696219247E26,0.0,0.0,1.0,0.0,4.0,0.8934641351750251,0.2520003165492412,0.999776481929382,0.42876304000000004,6.822069338039282E26
37935728745,0.7649251600000001,0.9990254682303955,0.0,0.0,0.0,0.0,7.453447996497709E15,1.182079318012817...,1.834087317424041...,1.0,1.0,1.0,4.444409355224051E-8,3.268008791003169...,0.08619800595860945,0.020506349875564236,1.032571255799820...,0.0,0.0,1.0,0.0,4.0,0.04250858184160272,0.02422476809978148,1.0,0.4546804899999999,1.216098195474771...
38700038932,1.276899999999994...,3.043338021093782E-4,9.879952219269599E34,4.561942969350385E37,1.589247853941863...,1.889970086499645E40,1.252427977239960...,6.606170420562422E41,2.606306585032650...,0.1481700512796872,0.42746499982132924,0.6193994207803898,0.7362937299586353,0.781995187252582,0.838392451455762,0.19101274188169637,2.184810796555127E44,0.0,0.0,1.0,0.0,4.0,0.993253282429309,0.30889816806781084,0.9952826595368763,0.9120250000000001,2.218937807472737...
41956465747,0.13097161,0.9998908878256969,9.752959721979141E14,3.416582495991781...,2.181445149981902...,9.412281567655927E20,8.739391954533618E21,4.263234806966908...,1.650426294502938...,0.06099307065171099,0.3412498035739627,0.8220611122288946,0.6693292178295999,0.7979427234164861,0.8423005331023574,0.16599133207626038,1.437109829501490...,0.0,0.0,1.0,0.0,4.0,0.5415404087099086,0.29152505097313